# Aula 2 - Parte Prática - Policy Gradients 

## Introdução

Nesse segundo notebook vamos aprender como melhorar o desempenho de um agente ao longo do tempo com base em experiências.

A abordagem que iremos estudar é conhecida por **Otimização de Política** baseada em gradientes (outro nome comum encontrado na literatura é Busca de Política (*Policy Search*)). De maneira geral, essa classe de algoritmos de RL é denominada por **Policy Gradients**.

Começaremos a explorar Policy Gradients nesse notebook com uma implementação básica. A fim de desenvolver gradualmente seus principais componentes e entender o papel de cada um deles, iremos de forma didática construir o algoritmo **REINFORCE**.

$$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left [ \sum_{t=0}^{T-1} \nabla_\theta \log \pi_\theta(\mathbf{a}_t|\mathbf{s}_t)  \sum_{t'=0}^{T-1} r_{t'} \right ]
$$


### Objetivos:

- Entender a abordagem de otimização de políticas como busca no espaço de parâmetros da política
- Implementar um primeiro agente baseado no algoritmo REINFORCE
- Familiarizar-se com a API básica de construção de modelos (i.e., redes neurais) em Keras
- Familiarizar-se com métodos de Deep Learning usando TensorFlow 2.X

### Imports

> **Atenção:** não se esqueça de executar todos os `imports` necessários antes prosseguir com o tutorial.

In [ ]:
import multiprocessing as mp

import gym
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

from utils.agent import RLAgent
from utils.memory import OnPolicyReplay
import utils.runner
from utils.viz import *

# sanity check
assert tf.__version__ == '2.1.0'
assert tf.executing_eagerly()

tf.get_logger().setLevel('ERROR') # ignore TensorFlow warnings

## 1. Redes Neurais em TensorFlow e Keras

Antes de começarmos propriamente com a implementação do algoritmo REINFORCE visto em aula. Iremos introduzir em linhas gerais a biblioteca de construção de modelos (i.e., redes neurai artificiais) do Keras no TensorFlow.

A grosso modo, a API do Keras é uma camada de *software* implementada em cima das funções, classes e métodos dos pacotes de Deep Learning com o objetivo de facilitar a implementação, treinameto e avaliação de redes neurais. Maiores detalhes sobre a especificação do Keras podem ser acessadas em [https://keras.io/](https://keras.io/). A documentação específica da implementação do Keras para TensorFlow se encontra em [https://www.tensorflow.org/guide/keras](https://www.tensorflow.org/guide/keras).

Note que a biblioteca do Keras é bastante extensa e flexível, de forma que nesse tutorial nos restringiremos a apresentar na medida do necessário os conceitos básicos necessários para trabalhar com redes neurais no contexto de Aprendizado por Reforço, e em particular, com foco nas primitivas necessárias para se implementar Policy Gradients.

> **Observação**: para aqueles interessados em uma introdução mais detalhada, sugerimos seguir após a aula os tutoriais do Tensorflow (e.g., https://www.tensorflow.org/tutorials/quickstart/beginner).


### 1.1 Camadas e Modelos Sequenciais

A rede neural mais simples que podemos construir é conhecida por *[Multi-Layer Perceptron](https://en.wikipedia.org/wiki/Multilayer_perceptron)* (MLP). Também conhecida por *Feedforward* ou *Fully-connected Network*.

Uma MLP é definida pela composição de camadas de transformações afins $h: \mathbb{R}^q \rightarrow \mathbb{R}^p$ seguidas por uma função não-linear (e.g., *activation*):
$
h(\mathbf{x}) = \text{activation}(\mathbf{W}_{p \times q} \cdot \mathbf{x}_{q \times 1} + \mathbf{b}_{p \times 1})
$ .

Recentemente, tem se tornado bastante comum o uso da função de ativação ReLu (*Rectified Linear Unit*):
$
\text{ReLu}(x) = max(0, x)
$ .

No Keras podemos utilizar a classe `tf.keras.layers.Dense` para construir a função $h(\cdot)$ e a classe `tf.keras.Sequential` para encadear essas transformações (comumente chamadas de *camadas*):

In [ ]:
mlp = tf.keras.Sequential([
    tf.keras.layers.Dense(32, input_shape=(5,), activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(3, activation="relu"),
])

In [ ]:
x = np.random.normal(size=(5,))
y = mlp(x[None,:])

print(f">> mlp(x) transforma vetores `x` de tamanho {x.shape[-1]} em vetores `y` de tamanho {y.shape[-1]}")

É importante observar que camadas e modelos em Keras esperam receber vetores em *batch*. Dessa forma, é possível processar conjuntos de vetores em paralelo; o que torna a computação consideravelmente mais eficiente.

No exemplo anterior, a variável `x = np.random.normal(size=(5,))` representa um único vetor de tamanho 5. Dessa forma, é necessário adicionar uma dimensão extra através do truque de manipulação de tensores `x[None,:]`.

In [ ]:
x = np.random.normal(size=(5,))
assert x.shape == (5,)
assert x[None,:].shape == (1, 5)

Para processar vetores em *batch*, o Keras assume que a primeira dimensão do vetor corresponde ao `batch_size`:

In [ ]:
batch_size = 128
x = np.random.normal(size=(batch_size, 5))
y = mlp(x)

print(f">> mlp(x) transformou {x.shape[0]} vetores `x` de tamanho {x.shape[-1]} em vetores {y.shape[0]} `y` de tamanho {y.shape[-1]}")

### 1.2 Camadas Probabilísticas

Em Aprendizado por Reforço é muitas vezes necessário representar distribuições (i.e., variáveis aleatórias). Por exemplo, na próxima seção utilizaremos uma distribuições para representar política estocásticas definida para um espaço discreto de ações.

Em TensorFlow 2.X é fácil representar as principais distribuições através da biblioteca `tensorflow_probability` (https://www.tensorflow.org/probability).

> **Observação**: para os interessados em modelos probabilísticos e inferência, recomendamos revisar após a aula o tutorial sobre regressão usando o TensorFlow Probability disponível em https://blog.tensorflow.org/2019/03/regression-with-probabilistic-layers-in.html.

Para esse tutorial precisaremos representar **distribuições categóricas** (https://en.wikipedia.org/wiki/Categorical_distribution) de forma a amostrar valoes discretos ou classes (representadas por números inteiros) de acordo com um vetor de probabilidades.

Lembre-se que para uma variável $x \in \{1, 2, \cdots, K\}$ e um vetor de probabilidades $[p_1, p_2, \cdots, p_K]$ (i.e., $p_i \in [0, 1], \sum_{i=1}^K p_i = 1$), temos que:

$$
x \sim \text{Categorical}([p_1, p_2, \cdots, p_K]) \Leftrightarrow P(x = i) = p_i
$$

No TensorFlow podemos adicionar um camada probabilística através da classe `tfp.layers.DistributionLambda`:

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, input_shape=(5,), activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(4),
    tfp.layers.DistributionLambda(lambda t: tfd.Categorical(logits=t))
])

Se invocarmos a rede neural implementada pela variável `model` em um conjunto de vetores de entrada `inputs`, observamos que a saída corresponde a um objeto do tipo `tfp.distributions.Categorical` e não a valores da variável aleatória:

In [ ]:
inputs = np.random.normal(size=(8, 5))

dist = model(inputs)
print(dist)

> **Atenção**: observe como o modelo representa um distribuição parametrizada (e.g., os parâmetros de uma categórica são conhecidos por`logits`). As primeiras camadas do tipo `Dense` processam os vetores de entrada gerando um conjunto de vetores de *logits* usados para definir a distribuição condicional categórica. Em outras palavras, para cada vetor de entrada temos uma distribuição categórica (por isso dizemos que o modelo representa uma distribuição condicional, isto é, condicionada no vetor de entrada.

Para amostrar realizações da variável aleatória dada pela distribuição categórica, utilizamos o método `sample()` (disponível para todas as distribuições definidas no TensorFlow Probability):

In [ ]:
sample = dist.sample()
print(sample)

> **Atenção**: note que embora a entrada do `model` seja um array n-dimensional no NumPy, uma amostra da distribuição é um objeto `Tensor` do TensorFlow. Você pode usar o método `numpy()` para transformar `tf.Tensor` em um `np.ndarray`.

Outro método importante para se familiarizar é `log_prob`. Dada uma amostra ou vetores de amostra, o método `log_prob` devolve o logaritmo da probabilidade de se ter gerado a amostra.

In [ ]:
sample_log_prob = dist.log_prob(sample)
print(sample_log_prob)

> **Atenção**: o método `log_prob` será fundamental na implementação dos algoritmos de *Policy Gradientes* nessse curso. Tenha certeza que você compreendeu seu funcionamento em termos de entradas e saídas da função.

## 2. Implementando políticas estocásticas em tf.Keras

Nesse tutorial iremos trabalhar com problemas do OpenAI gym que são definidos para um espaço discreto de ações. Dessa forma, precisaremos representar políticas estocásticas  $\pi_\theta(\mathbf{a}_t | \mathbf{s}_t)$ para ações discretas. Para isso, utilizaremos redes neurais construídas via Keras.

Para não perdermos tempo com detalhes de implementação, disponibilizamos a função `build_discrete_policy` que recebe as especificações dos espaços de observação e ação do ambiente e o hiper-parâmetros que definem a arquitetura da rede neural:

In [ ]:
def build_discrete_policy(obs_space, action_space, hidden_layers, activation="relu"):
    Input = tf.keras.Input
    Dense = tf.keras.layers.Dense
    DistributionLambda = tfp.layers.DistributionLambda
    Categorical = tfd.Categorical
    
    policy_net_layers = []

    policy_net_layers.append(Input(shape=obs_space.shape, name="State"))

    for i, units in enumerate(hidden_layers):
        policy_net_layers.append(Dense(units=units, activation=activation, name=f"Hidden{i+1}"))
    
    policy_net_layers.append(Dense(units=action_space.n, name="Logits"))
    policy_net_layers.append(DistributionLambda(lambda t: Categorical(logits=t), name="Action_Distribution_Categorical"))
                                     
    return tf.keras.Sequential(policy_net_layers)


Execute o código abaixo para criar um modelo com 2 camadas de 64 unidades utilizando o `relu` como função de ativação:

In [ ]:
env = gym.make("CartPole-v0")

hidden_layers = [64, 64]
activation = "relu"

policy = build_discrete_policy(env.observation_space, env.action_space, hidden_layers, activation)

obs = env.observation_space.sample()
action_dist = policy(obs[None,:])

action = action_dist.sample().numpy()
assert action[0] in env.action_space

Você pode inspecionar visualmente a arquitetura da política (e de qualquer modelo no Keras) através da função `tf.keras.utils.plot_model`:

In [ ]:
tf.keras.utils.plot_model(policy, show_shapes=True)

## 3. Agente REINFORCE

Uma vez que já nos famialiarizamos com a construção de modelos via Keras e com as particularidades de uma política estocástica implementada como modelo probabilística (via distribuição categórica), podemos começar a implementar a versão inicial do algoritmo *REINFORCE*. Lembre-se que, pela teoria desenvolvida em aula, o gradiente utilizado para otimizar a política é dado por:

$$
\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} \left [ \sum_{t=0}^{T-1} \nabla_\theta \log \pi_\theta(\mathbf{a}_t|\mathbf{s}_t) \hat{R}(\tau)\right ]~,
$$
onde $\hat{R}(\tau) = \sum_{t=0}^{T-1} r_t$ é retorno de uma trajetória simulada no ambiente
$
\tau = (\mathbf{s}_0, \mathbf{a}_0, r_0, \mathbf{s}_1, \mathbf{a}_1, r_1, \cdots, \mathbf{s}_{T-1}, \mathbf{a}_{T-1}, r_{T-1}) \sim \pi_\theta.
$

Como não é possível calcular analiticamente esse valor esperado no caso geral, estimaremos via **Monte-Carlo** o gradiente da política:

$$
\nabla_\theta J(\theta) \approx \frac{1}{N} \sum_{k=1}^K \sum_{t=0}^{T-1} \nabla_\theta \log \pi_\theta(\mathbf{a}_t^{(k)}|\mathbf{s}_t^{(k)}) \hat{R}(\tau^{(k)})~,
$$

onde o índice $k$ denota vetores da $k$-ésima trajetória amostrada. 

### 3.1 Surrogate Loss e diferenciação automática

Para usar a diferenciação automática no TensorFlow (i.e., computar gradientes via `tape.gradient`) é necessário definir uma função objetivo. No contexto de aprendizado de máquina no geral, essa função objetivo é tipicamente chamada de *loss* e a maioria dos pacotes de *Deep Learning* tenta **minimizar essa função**.

Dessa forma, como estamos querendo maximizar o desempenho do agente denotado por $J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta} [R(\tau)]$ deveremos minimizar o negativo de $J(\theta)$. Para o Policy Gradients a função de *loss* que vamos utilizar é dada por:

$$
\hat{J}(\theta) = \frac{1}{N} \sum_{k=1}^K \sum_{t=0}^{T-1}  \log \pi_\theta(\mathbf{a}_t^{(k)}|\mathbf{s}_t^{(k)}) \hat{R}(\tau^{(k)})~.
$$

> **IMPORTANTE**: note que em geral $\hat{J}(\theta) \neq J(\theta)$, no entanto, $\nabla_\theta \hat{J}(\theta) \approx \nabla_\theta J(\theta)$. Essa é a razão pela qual $\hat{J}$ é chamada de *surrogate loss*; *surrogate* em português significa "substituto". 

---

**<font color="red">EXERCÍCIO-PROGRAMA 1:</font>**

Nesse exercício você deverá codificar o método `REINFORCE._loss_fn` que será reponsável por implementar a função de *loss* do *Policy Gradient*.

> **Nota 1**: observe que cada `batch` corresponde a um dicionário de listas, onde cada lista corresponde a um conjunto de episódios. Por exemplo, `batch["states"][k]` devolve os estados do primeiro k-episódio amostrado.

> **Nota 2**: pode ser útil utilizar as funções de agregação `np.sum` e `tf.reduce_sum`.


In [ ]:
class REINFORCE(RLAgent):
    
    def __init__(self, obs_space, action_space, config=None):
        super(REINFORCE, self).__init__(obs_space, action_space, config)
        
        self.memory = OnPolicyReplay()
        self.policy = build_discrete_policy(self.obs_space, self.action_space, config["hidden_layers"], config["activation"])
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=config["learning_rate"])

    def act(self, obs):
        return self._act(obs).numpy()
    
    @tf.function
    def _act(self, obs):
        action_dist = self.policy(obs[None,:])
        return action_dist.sample()[0]

    def observe(self, obs, action, reward, next_obs, done):
        self.memory.update(obs, action, reward, next_obs, done)

    def learn(self):
        if self.memory.batch_size < self.config["train_batch_size"]:
            return
        
        batch = self.memory.sample()

        with tf.GradientTape() as tape:
            loss = self._loss_fn(batch)
            gradients = tape.gradient(loss, self.policy.trainable_weights)

        self.optimizer.apply_gradients(zip(gradients, self.policy.trainable_weights))
        
        grad_norm = [(w.name, tf.norm(grad).numpy()) for w, grad in zip(self.policy.trainable_weights, gradients)]
      
        return loss.numpy(), grad_norm

    def _loss_fn(self, batch):
        states, actions, rewards = batch["states"], batch["actions"], batch["rewards"]
        n_episodes = len(states)

        loss = 0.0
        
        # SEU CÓDIGO AQUI ===================================
        for episode in range(n_episodes):
            action_dist = self.policy(states[episode])
            log_probs = action_dist.log_prob(actions[episode])
            R = np.sum(rewards[episode])
            loss += - tf.reduce_sum(log_probs * R)
            
        loss /= n_episodes
        # ===================================================

        return loss

Para testar a sua implementação, rode o código abaixo. Para os parâmetros definidos abaixo no dicionário de `config`, você deve obter um `avg_loss` entre 350.0 e 450.0. Note que estamos usando simulação de Monte-Carlo então o resultado pode variar um pouco.

> **IMPORTANTE**: caso a sua implementação não esteja passando no teste abaixo (em várias tentativas consecutivas) e você achar que sua implementação está correta, chame o monitor ou um dos instrutores. 

In [ ]:
env = gym.make("CartPole-v0")

config = {
    # policy net
    "hidden_layers": [64, 64],
    "activation": "relu",

    # optimization
    "optimizer": "adam",
    "learning_rate": 1e-3,

    # training
    "train_batch_size": 3000,
}

agent = REINFORCE(env.observation_space, env.action_space, config=config)

avg_loss, trials = 0.0, 10
for j in range(trials):
    n_episodes = 100
    _ = utils.runner.evaluate(agent, env, n_episodes, render=False)
    loss = agent._loss_fn(agent.memory.sample())
    avg_loss += loss

avg_loss /= trials
    
assert 350.0 <= avg_loss <= 450.0, f"Você obteve avg_loss = {avg_loss}"
print(">> Sua implementação está provavelmente correta... Parabéns! :)")

## 4. Treinamento do agente

In [ ]:
env = gym.make("CartPole-v0")

config = {
    # policy net
    "hidden_layers": [64, 64],
    "activation": "relu",

    # optimization
    "optimizer": "adam",
    "learning_rate": 1e-3,

    # training
    "train_batch_size": 3000,
}

agent = REINFORCE(env.observation_space, env.action_space, config=config)

In [ ]:
n_episodes = 100
timesteps, total_rewards, avg_total_rewards = utils.runner.evaluate(agent, env, n_episodes, render=False)
plot_action_distribution(agent)

In [ ]:
n_episodes = 3
_ = utils.runner.evaluate(agent, env, n_episodes, render=True)

In [ ]:
total_timesteps = 1_000_000
timesteps, losses, grads, total_rewards, avg_total_rewards = utils.runner.train(agent, env, total_timesteps)

In [ ]:
plot_returns(timesteps, total_rewards, avg_total_rewards)

In [ ]:
plot_losses(losses)

In [ ]:
plot_gradients(grads)

In [ ]:
n_episodes = 100
timesteps, total_rewards, avg_total_rewards = utils.runner.evaluate(agent, env, n_episodes, render=False)
plot_action_distribution(agent)

In [ ]:
n_episodes = 5
_ = utils.runner.evaluate(agent, env, n_episodes, render=True)

## 5. Experimentos 

In [ ]:
def run_experiment(trial, env_id, config, total_timesteps):
    env = gym.make(env_id)
    agent = REINFORCE(env.observation_space, env.action_space, config)
    timesteps, losses, _, total_rewards, avg_total_rewards = utils.runner.train(agent, env, total_timesteps, verbose=True)
    return timesteps, losses, total_rewards, avg_total_rewards


In [ ]:
n_trials = 3
total_timesteps = 1_000_000

env_id = "CartPole-v0"
base_config = {
        # policy net
    "hidden_layers": [64, 64],
    "activation": "relu",

    # optimization
    "optimizer": "adam",
    "learning_rate": None,

    # training
    "train_batch_size": None,
}

learning_rates = [1e-2, 1e-4]
batch_sizes = [1000, 10000]

for lr in learning_rates:
    for bs in batch_sizes:        
        experiment_config = {"learning_rate": lr, "train_batch_size": bs}
        config = {**base_config, **experiment_config}
        
        print(f">> Experiment: {experiment_config}")
        results = []
        for trial in range(n_trials):
            print(f"Starting trial {trial+1} ...")
            results.append(run_experiment(trial, env_id, config, total_timesteps))
        
        plot_experiments(results, experiment_config)